In [14]:
import os, math
import numpy as np
import pandas as pd
import seaborn as sns
import sys
sys.path.append('/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/yao/misc')
import helpers

%matplotlib inline
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, f_classif
from sklearn.metrics import log_loss

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV


In [2]:
train_full = pd.read_csv('../feature_engineer/training_SeparateText.csv')
train_full = train_full.sort_values('ID').reset_index(drop=True)

Use TF-IDF to Vectorize the texts， a feature selector, then SVM one-vs-all classification. 

In [15]:
##Pipeline
tfidf = TfidfVectorizer(max_features=50000, lowercase = True, analyzer='word', max_df=0.7, min_df=1)
ffilter = TruncatedSVD(n_components=1000)

##Data and labels
y_train = train_full["Class"]-1
X_train = ffilter.fit_transform(tfidf.fit_transform(train_full["SeparateText"]), y_train)
#X_train = preprocessing.normalize(X_train, norm='l2')

#y_val = val_full["Class"]-1
#X_val = ffilter.transform(tfidf.transform(val_full["Text"]))

#y_test = test_full["Class"]-1
#X_test = ffilter.transform(tfidf.transform(test_full["Text"]))

#y_train_bi = label_binarize(y_train, classes=range(9))
#y_val_bi = label_binarize(y_val, classes=range(9))
#y_test_bi = label_binarize(y_test, classes=range(9))


In [16]:
##Fitting
parameters = {
    "estimator__C": [0.05],
    "estimator__kernel": ['linear']
    #"estimator__degree": [2, 3]
}

clf = GridSearchCV(OneVsRestClassifier(svm.SVC(probability=True, class_weight='balanced')), cv = 3, param_grid=parameters, scoring='neg_log_loss', n_jobs=-1)

clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'estimator__C': [0.05], 'estimator__kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [13]:
pd.DataFrame.from_dict(clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_estimator__C,param_estimator__kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,284.947069,21.812883,-1.647345,-0.831966,0.05,linear,"{'estimator__C': 0.05, 'estimator__kernel': 'l...",1,-1.708772,-0.807363,-1.64562,-0.818296,-1.587149,-0.87024,5.275745,0.61347,0.049656,0.027429


In [26]:
#Evaluate
y_train_prob = clf.predict_proba(X_train)
#y_test_prob = clf.predict_proba(X_test)
print(log_loss(y_train, y_train_prob, eps=1e-15, normalize=True, labels=range(9)))
#print(log_loss(y_val, clf.predict_proba(X_val), eps=1e-15, normalize=True, labels=range(9)))
#print(log_loss(y_test, y_test_prob, eps=1e-15, normalize=True, labels=range(9)))

1.22687010854


In [ ]:
#submission('./2nd_layer_data/train_prob', y_train_prob)
#submission('./2nd_layer_data/test_prob', y_test_prob)

In [ ]:
y_train

In [ ]:
helpers.plot_roc_curve(y_test_bi, y_test_prob)

In [8]:
start_time = time.time()


X_submit = ffilter.transform(tfidf.transform(\
    pd.read_csv('./input/test_text', sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])["Text"]))

y_submit_prob = clf.predict_proba(X_submit) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 231.57416605949402 seconds ---


In [9]:
helpers.submission('./2nd_layer_data/true_test_prob', y_submit_prob)
np.save('./2nd_layer_data/X_true_test_tfidf', X_submit)

In [ ]:
X_train_tfidf = np.array(X_train.todense())
X_test_tfidf = np.array(X_test.todense())

In [ ]:
np.save('./2nd_layer_data/X_train_tfidf', X_train_tfidf)
np.save('./2nd_layer_data/X_test_tfidf', X_test_tfidf)
np.save('./2nd_layer_data/y_train_tfidf', y_train_bi)
np.save('./2nd_layer_data/y_test_tfidf', y_test_bi)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
1.10161488021
1.19078777039